# MNIST end to end on Kubeflow on GKE ( Using Katib)

This example guides you through:
  
  1. Taking an example TensorFlow model and modifying it to support distributed training.
  1. Serving the resulting model using TFServing.
  1. Deploying and using a web app that sends prediction requests to the model.
  
## Requirements

  * You must be running Kubeflow 1.0 on Kubernetes Engine (GKE) with Cloud Identity-Aware Proxy (Cloud IAP).
  * Run this notebook within your Kubeflow cluster. See the guide to [setting up your Kubeflow notebooks](https://www.kubeflow.org/docs/notebooks/setup/).
 

### Prepare model

There is a delta between existing distributed MNIST examples and what's needed to run well as a TFJob.

Basically, you must:

* Add options in order to make the model configurable.
* Use `tf.estimator.train_and_evaluate` to enable model exporting and serving.
* Define serving signatures for model serving.

This tutorial provides a Python program that's already prepared for you: [model.py](model.py).

### Change Directory to Main Katib folder

In [1]:
cd /root/katib/sdk/python

/root/katib/sdk/python


### Install Katib SDK

In [2]:
!python setup.py install

### Import Required Libraries for Katib usage

In [5]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container

In [6]:
import kubeflow.katib as kc
from kubeflow.katib import constants
from kubeflow.katib import utils
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSetting
from kubeflow.katib import V1alpha3AlgorithmSpec
from kubeflow.katib import V1alpha3CollectorSpec
from kubeflow.katib import V1alpha3EarlyStoppingSetting
from kubeflow.katib import V1alpha3EarlyStoppingSpec
from kubeflow.katib import V1alpha3Experiment
from kubeflow.katib import V1alpha3ExperimentCondition
from kubeflow.katib import V1alpha3ExperimentList
from kubeflow.katib import V1alpha3ExperimentSpec
from kubeflow.katib import V1alpha3ExperimentStatus
from kubeflow.katib import V1alpha3FeasibleSpace
from kubeflow.katib import V1alpha3FileSystemPath
from kubeflow.katib import V1alpha3FilterSpec
from kubeflow.katib import V1alpha3GoTemplate
from kubeflow.katib import V1alpha3GraphConfig
from kubeflow.katib import V1alpha3Metric
from kubeflow.katib import V1alpha3MetricsCollectorSpec
from kubeflow.katib import V1alpha3NasConfig
from kubeflow.katib import V1alpha3ObjectiveSpec
from kubeflow.katib import V1alpha3Observation
from kubeflow.katib import V1alpha3Operation
from kubeflow.katib import V1alpha3OptimalTrial
from kubeflow.katib import V1alpha3ParameterAssignment
from kubeflow.katib import V1alpha3ParameterSpec
from kubeflow.katib import V1alpha3SourceSpec
from kubeflow.katib import V1alpha3Suggestion
from kubeflow.katib import V1alpha3SuggestionCondition
from kubeflow.katib import V1alpha3SuggestionList
from kubeflow.katib import V1alpha3SuggestionSpec
from kubeflow.katib import V1alpha3SuggestionStatus
from kubeflow.katib import V1alpha3TemplateSpec
from kubeflow.katib import V1alpha3Trial
from kubeflow.katib import V1alpha3TrialAssignment
from kubeflow.katib import V1alpha3TrialCondition
from kubeflow.katib import V1alpha3TrialList
from kubeflow.katib import V1alpha3TrialSpec
from kubeflow.katib import V1alpha3TrialStatus
from kubeflow.katib import V1alpha3TrialTemplate

### Verify that you have a Google Cloud Platform (GCP) account

The cell below checks that this notebook was spawned with credentials to access GCP.


In [7]:
import logging
import os
import uuid
from importlib import reload
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()

In [8]:
cd /root/examples/mnist

/root/examples/mnist


### Install the required libraries

Run the next cell to import the libraries required to train this model.

In [9]:
import notebook_setup
reload(notebook_setup)
notebook_setup.notebook_setup()

pip installing requirements.txt
Checkout kubeflow/tf-operator @9238906
Adding /root/git_tf-operator/sdk/python to python path
Configure docker credentials


Wait for the message `Configure docker credentials` before moving on to the next cell.

In [10]:
import k8s_util
# Force a reload of Kubeflow. Since Kubeflow is a multi namespace module,
# doing the reload in notebook_setup may not be sufficient.
import kubeflow
reload(kubeflow)
from kubernetes import client as k8s_client
from kubernetes import config as k8s_config
from kubeflow.tfjob.api import tf_job_client as tf_job_client_module
from IPython.core.display import display, HTML
import yaml

## Configure a Docker registry for Kubeflow Fairing

* In order to build Docker images from your notebook, you need a Docker registry to store the images.
* Below you set some variables specifying a [Container Registry](https://cloud.google.com/container-registry/docs/).
* Kubeflow Fairing provides a utility function to guess the name of your GCP project.

In [11]:
from kubernetes import client as k8s_client
from kubernetes.client import rest as k8s_rest
from kubeflow import fairing   
from kubeflow.fairing import utils as fairing_utils
from kubeflow.fairing.builders import append
from kubeflow.fairing.deployers import job
from kubeflow.fairing.preprocessors import base as base_preprocessor

# Setting up Google Container Registry (GCR) for storing output containers.
# You can use any Docker container registry instead of GCR.
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(GCP_PROJECT)
namespace = fairing_utils.get_current_k8s_namespace()

logging.info(f"Running in project {GCP_PROJECT}")
logging.info(f"Running in namespace {namespace}")
logging.info(f"Using Docker registry {DOCKER_REGISTRY}")


Running in project cpsg-ai-kubeflow
Running in namespace anonymous
Using Docker registry gcr.io/cpsg-ai-kubeflow/fairing-job


## Use Kubeflow Fairing to build the Docker image

This notebook uses Kubeflow Fairing's kaniko builder to build a Docker image that includes all your dependencies.
  * You use kaniko because you want to be able to run `pip` to install dependencies.
  * Kaniko gives you the flexibility to build images from Dockerfiles.

In [12]:
# TODO(https://github.com/kubeflow/fairing/issues/426): We should get rid of this once the default 
# Kaniko image is updated to a newer image than 0.7.0.
from kubeflow.fairing import constants
constants.constants.KANIKO_IMAGE = "gcr.io/kaniko-project/executor:v0.14.0"

In [13]:
from kubeflow.fairing.builders import cluster

# output_map is a map of extra files to add to the notebook.
# It is a map from source location to the location inside the context.
output_map =  {
    "Dockerfile.model": "Dockerfile",
    "model.py": "model.py"
}


preprocessor = base_preprocessor.BasePreProcessor(
    command=["python"], # The base class will set this.
    input_files=[],
    path_prefix="/app", # irrelevant since we aren't preprocessing any files
    output_map=output_map)

preprocessor.preprocess()

set()

Run the next cell and wait until you see a message like `Built image gcr.io/<your-project>/fairing-job/mnist:<1234567>`.

In [14]:
# Use a Tensorflow image as the base image
# We use a custom Dockerfile 
cluster_builder = cluster.cluster.ClusterBuilder(registry=DOCKER_REGISTRY,
                                                 base_image="", # base_image is set in the Dockerfile
                                                 preprocessor=preprocessor,
                                                 image_name="mnist",
                                                 dockerfile_path="Dockerfile",
                                                 pod_spec_mutators=[fairing.cloud.gcp.add_gcp_credentials_if_exists],
                                                 context_source=cluster.gcs_context.GCSContextSource())
cluster_builder.build()
logging.info(f"Built image {cluster_builder.image_tag}")

Building image using cluster builder.
Creating docker context: /tmp/fairing_context_9ndd9jei
Dockerfile already exists in Fairing context, skipping...
Not able to find gcp credentials secret: user-gcp-sa
Trying workload identity service account: default-editor
Waiting for fairing-builder-tqqlf-bng99 to start...
Waiting for fairing-builder-tqqlf-bng99 to start...
Waiting for fairing-builder-tqqlf-bng99 to start...
Pod started running True


INFO[0003] Resolved base name tensorflow/tensorflow:1.15.2-py3 to tensorflow/tensorflow:1.15.2-py3
INFO[0003] Resolved base name tensorflow/tensorflow:1.15.2-py3 to tensorflow/tensorflow:1.15.2-py3
INFO[0003] Downloading base image tensorflow/tensorflow:1.15.2-py3
ERROR: logging before flag.Parse: E0331 10:49:27.227715       1 metadata.go:142] while reading 'google-dockercfg' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg
ERROR: logging before flag.Parse: E0331 10:49:27.232442       1 metadata.go:159] while reading 'google-dockercfg-url' metadata: http status code: 404 while fetching url http://metadata.google.internal./computeMetadata/v1/instance/attributes/google-dockercfg-url
INFO[0006] Error while retrieving image from cache: getting file info: stat /cache/sha256:28b5f547969d70f825909c8fe06675ffc2959afe6079aeae754afa312f6417b9: no such file or directory
INFO[0006] Downloading base image ten

Built image gcr.io/cpsg-ai-kubeflow/fairing-job/mnist:F073324E


## Create a Cloud Storage bucket

Run the next cell to create a Google Cloud Storage (GCS) bucket to store your models and other results.

Since this notebook is running in Python, the cell uses the GCS Python client libraries, but you can use the `gsutil` command line instead.

In [15]:
from google.cloud import storage
bucket = f"{GCP_PROJECT}-mnist"

client = storage.Client()
b = storage.Bucket(client=client, name=bucket)

if not b.exists():
    logging.info(f"Creating bucket {bucket}")
    b.create()
else:
    logging.info(f"Bucket {bucket} already exists")    

Bucket cpsg-ai-kubeflow-mnist already exists


### Define Experiment related Specs

In [16]:
algorithmsettings = V1alpha3AlgorithmSetting(
    name= "random_state",
    value = "10"
    )
algorithm = V1alpha3AlgorithmSpec(
    algorithm_name = "bayesianoptimization",
    algorithm_settings = [algorithmsettings]
  )

# Metric Collector
collector = V1alpha3CollectorSpec(kind = "TensorFlowEvent")
FileSystemPath = V1alpha3FileSystemPath(kind = "/train" , path = "Directory")
metrics_collector_spec = V1alpha3MetricsCollectorSpec(
    collector = collector,
    source = FileSystemPath)

# Objective
objective = V1alpha3ObjectiveSpec(
    goal = 0.9999,
    objective_metric_name = "Validation-accuracy",
    type = "maximize")

# Parameters

feasible_space_optimizer = V1alpha3FeasibleSpace(list = ["sgd", "adam", "ftrl"])
feasible_space_lr = V1alpha3FeasibleSpace(min = "0.01", max = "0.03")
feasible_space_layers = V1alpha3FeasibleSpace(min = "2", max = "5")

parameters = [V1alpha3ParameterSpec(
    feasible_space = feasible_space_optimizer, 
    name = "--optimizer",
    parameter_type = "categorical"
    ),
    V1alpha3ParameterSpec(
    feasible_space = feasible_space_lr, 
    name = "--lr",
    parameter_type ="double"
    ),
    V1alpha3ParameterSpec(
    feasible_space = feasible_space_layers, 
    name = "--num-layers",
    parameter_type = "int"
    )]

# Trialtemplate
go_template = V1alpha3GoTemplate(
    raw_template =   "apiVersion: \"batch/v1\"\nkind: Job\nmetadata:\n  name: {{.Trial}}\n  namespace: {{.NameSpace}}\nspec:\n  template:\n    spec:\n      containers:\n      - name: {{.Trial}}\n        image: docker.io/kubeflowkatib/mxnet-mnist\n        command:\n        - \"python3\"\n        - \"/opt/mxnet-mnist/mnist.py\"\n        - \"--batch-size=64\"\n        {{- with .HyperParameters}}\n        {{- range .}}\n        - \"{{.Name}}={{.Value}}\"\n        {{- end}}\n        {{- end}}\n      restartPolicy: Never"
    )


trial_template= V1alpha3TrialTemplate(go_template=go_template)


# Experiment
experiment = V1alpha3Experiment(
    api_version="kubeflow.org/v1alpha3",
    kind="Experiment",
    metadata=V1ObjectMeta(name="bayesianoptimizationjob",namespace="anonymous"),

    spec=V1alpha3ExperimentSpec(
         algorithm = algorithm,
         max_failed_trial_count=3,
         max_trial_count=5,
         objective = objective,
         parallel_trial_count=5,
         parameters = parameters ,
         trial_template = trial_template
    )
)

### Define default namespace

In [17]:
namespace = kc.utils.get_default_target_namespace()

In [18]:
kclient = kc.KatibClient()

### Create Experiment

In [19]:
kclient.create_experiment(experiment, namespace=namespace)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-03-31T10:50:33Z',
  'generation': 1,
  'name': 'bayesianoptimizationjob',
  'namespace': 'anonymous',
  'resourceVersion': '19337136',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/anonymous/experiments/bayesianoptimizationjob',
  'uid': '7258cc60-733d-11ea-bedf-42010aa00fec'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{'name': 'random_state', 'value': '10'}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 5,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.9999,
   'objectiveMetricName': 'Validation-accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 5,
  'parameters': [{'feasibleSpace': {'list': ['sgd', 'adam', 'ftrl']},
    'name': '--optimizer',
    'parameterType': 'categorical'},
   {'feasibleSpace': {'max': '0.03', 'min': '0.01'},
    'name': '--lr',
    'parameterType': 'doub

### Get Single Experiment Details

In [20]:
kclient.get_experiment(name="bayesianoptimizationjob", namespace=None)

{'apiVersion': 'kubeflow.org/v1alpha3',
 'kind': 'Experiment',
 'metadata': {'creationTimestamp': '2020-03-31T10:50:33Z',
  'finalizers': ['update-prometheus-metrics'],
  'generation': 1,
  'name': 'bayesianoptimizationjob',
  'namespace': 'anonymous',
  'resourceVersion': '19337138',
  'selfLink': '/apis/kubeflow.org/v1alpha3/namespaces/anonymous/experiments/bayesianoptimizationjob',
  'uid': '7258cc60-733d-11ea-bedf-42010aa00fec'},
 'spec': {'algorithm': {'algorithmName': 'bayesianoptimization',
   'algorithmSettings': [{'name': 'random_state', 'value': '10'}]},
  'maxFailedTrialCount': 3,
  'maxTrialCount': 5,
  'metricsCollectorSpec': {'collector': {'kind': 'StdOut'}},
  'objective': {'goal': 0.9999,
   'objectiveMetricName': 'Validation-accuracy',
   'type': 'maximize'},
  'parallelTrialCount': 5,
  'parameters': [{'feasibleSpace': {'list': ['sgd', 'adam', 'ftrl']},
    'name': '--optimizer',
    'parameterType': 'categorical'},
   {'feasibleSpace': {'max': '0.03', 'min': '0.01'},

### Get Experiment status

In [29]:
kclient.get_experiment_status(name="bayesianoptimizationjob", namespace=namespace)

'Succeeded'

### Check whether experiment has succeeded

In [30]:
kclient.is_experiment_succeeded(name="bayesianoptimizationjob", namespace=namespace)

True

### List Trials of Experiment

In [28]:
kclient.list_trials(name="bayesianoptimizationjob", namespace=namespace)

[{'name': 'bayesianoptimizationjob-4h9cs256', 'status': 'Succeeded'},
 {'name': 'bayesianoptimizationjob-8ngjhvt5', 'status': 'Succeeded'},
 {'name': 'bayesianoptimizationjob-gzw8lfnq', 'status': 'Succeeded'},
 {'name': 'bayesianoptimizationjob-jtpbl78z', 'status': 'Succeeded'},
 {'name': 'bayesianoptimizationjob-rvpllnx4', 'status': 'Succeeded'}]

### List all experiments

In [31]:
kclient.list_experiments(namespace=namespace)


[{'name': 'bayesianoptimizationjob', 'status': 'Succeeded'}]

### Get & Print the optimal Hyperparameters

In [32]:
optimal_lr = kclient.get_optimal_hyperparmeters(name="bayesianoptimizationjob", namespace=namespace)
print(optimal_lr)

{'status': 'Succeeded', 'currentOptimalTrial': {'bestTrialName': 'bayesianoptimizationjob-gzw8lfnq', 'observation': {'metrics': [{'name': 'Validation-accuracy', 'value': 0.979697}]}, 'parameterAssignments': [{'name': '--optimizer', 'value': 'sgd'}, {'name': '--lr', 'value': '0.01721786775240965'}, {'name': '--num-layers', 'value': '3'}]}}


### Optimal Value of Learning Rate

In [33]:
optimal_lr['currentOptimalTrial']['parameterAssignments'][1]['value']

'0.01721786775240965'

## Distributed training

To train the model, this example uses [TFJob](https://www.kubeflow.org/docs/components/training/tftraining/) to run a distributed training job. Run the next cell to set up the YAML specification for the job:

In [34]:
train_name = f"mnist-train-{uuid.uuid4().hex[:4]}"
num_ps = 1
num_workers = 2
model_dir = f"gs://{bucket}/mnist"
export_path = f"gs://{bucket}/mnist/export" 
train_steps = 200
batch_size = 100
learning_rate = optimal_lr['currentOptimalTrial']['parameterAssignments'][1]['value']
image = cluster_builder.image_tag

train_spec = f"""apiVersion: kubeflow.org/v1
kind: TFJob
metadata:
  name: {train_name}  
spec:
  tfReplicaSpecs:
    Ps:
      replicas: {num_ps}
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          serviceAccount: default-editor
          containers:
          - name: tensorflow
            command:
            - python
            - /opt/model.py
            - --tf-model-dir={model_dir}
            - --tf-export-dir={export_path}
            - --tf-train-steps={train_steps}
            - --tf-batch-size={batch_size}
            - --tf-learning-rate={learning_rate}
            image: {image}
            workingDir: /opt
          restartPolicy: OnFailure
    Chief:
      replicas: 1
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          serviceAccount: default-editor
          containers:
          - name: tensorflow
            command:
            - python
            - /opt/model.py
            - --tf-model-dir={model_dir}
            - --tf-export-dir={export_path}
            - --tf-train-steps={train_steps}
            - --tf-batch-size={batch_size}
            - --tf-learning-rate={learning_rate}
            image: {image}
            workingDir: /opt
          restartPolicy: OnFailure
    Worker:
      replicas: 1
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          serviceAccount: default-editor
          containers:
          - name: tensorflow
            command:
            - python
            - /opt/model.py
            - --tf-model-dir={model_dir}
            - --tf-export-dir={export_path}
            - --tf-train-steps={train_steps}
            - --tf-batch-size={batch_size}
            - --tf-learning-rate={learning_rate}
            image: {image}
            workingDir: /opt
          restartPolicy: OnFailure
"""           

### Create the training job

To submit the training job, you could write the spec to a YAML file and then do `kubectl apply -f {FILE}`.

However, because you are running in a Jupyter notebook, you use the TFJob client. 
* You run the TFJob in a namespace created by a Kubeflow profile.
* The namespace is the same as the namespace where you are running the notebook.
* Creating a profile ensures that the namespace is provisioned with service accounts and other resources needed for Kubeflow.

In [35]:
tf_job_client = tf_job_client_module.TFJobClient()

In [36]:
tf_job_body = yaml.safe_load(train_spec)
tf_job = tf_job_client.create(tf_job_body, namespace=namespace)  

logging.info(f"Created job {namespace}.{train_name}")

Created job anonymous.mnist-train-1dcb


### Check the job using kubectl

Above you used the Python SDK for TFJob to check the status. You can also use kubectl get the status of your job. 
The job conditions will tell you whether the job is running, succeeded or failed.

In [37]:
!kubectl get tfjobs -o yaml {train_name}

apiVersion: kubeflow.org/v1
kind: TFJob
metadata:
  creationTimestamp: "2020-03-31T10:59:14Z"
  generation: 1
  name: mnist-train-1dcb
  namespace: anonymous
  resourceVersion: "19341273"
  selfLink: /apis/kubeflow.org/v1/namespaces/anonymous/tfjobs/mnist-train-1dcb
  uid: a89458cf-733e-11ea-bedf-42010aa00fec
spec:
  tfReplicaSpecs:
    Chief:
      replicas: 1
      template:
        metadata:
          annotations:
            sidecar.istio.io/inject: "false"
        spec:
          containers:
          - command:
            - python
            - /opt/model.py
            - --tf-model-dir=gs://cpsg-ai-kubeflow-mnist/mnist
            - --tf-export-dir=gs://cpsg-ai-kubeflow-mnist/mnist/export
            - --tf-train-steps=200
            - --tf-batch-size=100
            - --tf-learning-rate=0.01721786775240965
            image: gcr.io/cpsg-ai-kubeflow/fairing-job/mnist:F073324E
            name: tensorflow
            workingDir: /opt
          restartPolicy: OnFailure
         

### Get the training logs

* There are two ways to get the logs for the training job:

  * Using kubectl to fetch the pod logs. These logs are ephemeral; they will be unavailable when the pod is garbage collected to free up resources.
  * Using Stackdriver.
  
     * Kubernetes logs are automatically available in Stackdriver.
     * You can use labels to locate the logs for a specific pod.
     * In the cell below, you use labels for the training job name and process type to locate the logs for a specific pod.
  
* Run the cell below to get a link to Stackdriver for your logs:

In [38]:
from urllib.parse import urlencode

for replica in ["chief", "worker", "ps"]:    
    logs_filter = f"""resource.type="k8s_container"    
    labels."k8s-pod/tf-job-name" = "{train_name}"
    labels."k8s-pod/tf-replica-type" = "{replica}"    
    resource.labels.container_name="tensorflow" """

    new_params = {'project': GCP_PROJECT,
                  # Logs for last 7 days
                  'interval': 'P7D',
                  'advancedFilter': logs_filter}

    query = urlencode(new_params)

    url = "https://console.cloud.google.com/logs/viewer?" + query

    display(HTML(f"Link to: <a href='{url}'>{replica} logs</a>"))


## Deploy TensorBoard

The next step is to create a Kubernetes deployment to run TensorBoard.

TensorBoard will be accessible behind the Kubeflow IAP endpoint.

In [39]:
tb_name = "mnist-tensorboard"
tb_deploy = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: mnist-tensorboard
  name: {tb_name}
  namespace: {namespace}
spec:
  selector:
    matchLabels:
      app: mnist-tensorboard
  template:
    metadata:
      labels:
        app: mnist-tensorboard
        version: v1
    spec:
      serviceAccount: default-editor
      containers:
      - command:
        - /usr/local/bin/tensorboard
        - --logdir={model_dir}
        - --port=80
        - --host=0.0.0.0
        
        image: tensorflow/tensorflow:1.15.2-py3
        name: tensorboard
        ports:
        - containerPort: 80
"""
tb_service = f"""apiVersion: v1
kind: Service
metadata:
  labels:
    app: mnist-tensorboard
  name: {tb_name}
  namespace: {namespace}
spec:
  ports:
  - name: http-tb
    port: 80
    targetPort: 80
  selector:
    app: mnist-tensorboard
  type: ClusterIP
"""

tb_virtual_service = f"""apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: {tb_name}
  namespace: {namespace}
spec:
  gateways:
  - kubeflow/kubeflow-gateway
  hosts:
  - '*'
  http:
  - match:
    - uri:
        prefix: /mnist/{namespace}/tensorboard/
    rewrite:
      uri: /
    route:
    - destination:
        host: {tb_name}.{namespace}.svc.cluster.local
        port:
          number: 80
    timeout: 300s
"""

tb_specs = [tb_deploy, tb_service, tb_virtual_service]

In [40]:
k8s_util.apply_k8s_specs(tb_specs, k8s_util.K8S_CREATE_OR_REPLACE)

/root/examples/mnist/k8s_util.py:55: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  spec = yaml.load(spec)
Created Deployment anonymous.mnist-tensorboard
Created Service anonymous.mnist-tensorboard
Deleted VirtualService anonymous.mnist-tensorboard
Created VirtualService mnist-tensorboard.mnist-tensorboard


[{'api_version': 'apps/v1',
  'kind': 'Deployment',
  'metadata': {'annotations': None,
               'cluster_name': None,
               'creation_timestamp': datetime.datetime(2020, 3, 31, 10, 59, 33, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': 1,
               'initializers': None,
               'labels': {'app': 'mnist-tensorboard'},
               'managed_fields': None,
               'name': 'mnist-tensorboard',
               'namespace': 'anonymous',
               'owner_references': None,
               'resource_version': '19341380',
               'self_link': '/apis/apps/v1/namespaces/anonymous/deployments/mnist-tensorboard',
               'uid': 'b44d2e34-733e-11ea-bedf-42010aa00fec'},
  'spec': {'min_ready_seconds': None,
           'paused': None,
           'progress_deadline_seconds': 600,
 

## Set a variable defining your endpoint

Set `endpoint` to `https://your-domain` (with no slash at the end). You can see your domain in the URL that you're using to access this notebook.

In [41]:
endpoint = "http://35.200.222.152:31380"

if endpoint:
  logging.info(f"endpoint set to {endpoint}")
else:
  logging.info("Warning: You must set {endpoint} in order to print out the URLs where you can access your web apps.")

endpoint set to http://35.200.222.152:31380


## Access the TensorBoard UI

Run the cell below to find the endpoint for the TensorBoard UI.

In [42]:
if endpoint:    
    vs = yaml.safe_load(tb_virtual_service)
    path= vs["spec"]["http"][0]["match"][0]["uri"]["prefix"]
    tb_endpoint = endpoint + path
    display(HTML(f"TensorBoard UI is at <a href='{tb_endpoint}'>{tb_endpoint}</a>"))

## Wait for the training job to finish

You can use the TFJob client to wait for the job to finish:

In [43]:
tf_job = tf_job_client.wait_for_condition(train_name, expected_condition=["Succeeded", "Failed"], namespace=namespace)

if tf_job_client.is_job_succeeded(train_name, namespace):
    logging.info(f"TFJob {namespace}.{train_name} succeeded")
else:
    raise ValueError(f"TFJob {namespace}.{train_name} failed")  

TFJob anonymous.mnist-train-1dcb succeeded


## Serve the model

Now you can deploy the model using [TensorFlow Serving](https://www.kubeflow.org/docs/components/serving/tfserving_new/).

You need to create the following:
* A Kubernetes deployment.
* A Kubernetes service.
* (Optional) A configmap containing the Prometheus monitoring configuration.

In [44]:
deploy_name = "mnist-model"
model_base_path = export_path

# The web UI defaults to mnist-service so if you change the name, you must
# change it in the UI as well.
model_service = "mnist-service"

deploy_spec = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  labels:
    app: mnist
  name: {deploy_name}
  namespace: {namespace}
spec:
  selector:
    matchLabels:
      app: mnist-model
  template:
    metadata:
      # TODO(jlewi): Right now we disable the istio side car because otherwise ISTIO rbac will prevent the
      # UI from sending RPCs to the server. We should create an appropriate ISTIO rbac authorization
      # policy to allow traffic from the UI to the model servier.
      # https://istio.io/docs/concepts/security/#target-selectors
      annotations:        
        sidecar.istio.io/inject: "false"
      labels:
        app: mnist-model
        version: v1
    spec:
      serviceAccount: default-editor
      containers:
      - args:
        - --port=9000
        - --rest_api_port=8500
        - --model_name=mnist
        - --model_base_path={model_base_path}
        - --monitoring_config_file=/var/config/monitoring_config.txt
        command:
        - /usr/bin/tensorflow_model_server
        env:
        - name: modelBasePath
          value: {model_base_path}
        image: tensorflow/serving:1.15.0
        imagePullPolicy: IfNotPresent
        livenessProbe:
          initialDelaySeconds: 30
          periodSeconds: 30
          tcpSocket:
            port: 9000
        name: mnist
        ports:
        - containerPort: 9000
        - containerPort: 8500
        resources:
          limits:
            cpu: "4"
            memory: 4Gi
          requests:
            cpu: "1"
            memory: 1Gi
        volumeMounts:
        - mountPath: /var/config/
          name: model-config
      volumes:
      - configMap:
          name: {deploy_name}
        name: model-config
"""

service_spec = f"""apiVersion: v1
kind: Service
metadata:
  annotations:    
    prometheus.io/path: /monitoring/prometheus/metrics
    prometheus.io/port: "8500"
    prometheus.io/scrape: "true"
  labels:
    app: mnist-model
  name: {model_service}
  namespace: {namespace}
spec:
  ports:
  - name: grpc-tf-serving
    port: 9000
    targetPort: 9000
  - name: http-tf-serving
    port: 8500
    targetPort: 8500
  selector:
    app: mnist-model
  type: ClusterIP
"""

monitoring_config = f"""kind: ConfigMap
apiVersion: v1
metadata:
  name: {deploy_name}
  namespace: {namespace}
data:
  monitoring_config.txt: |-
    prometheus_config: {{
      enable: true,
      path: "/monitoring/prometheus/metrics"
    }}
"""

model_specs = [deploy_spec, service_spec, monitoring_config]

In [45]:
k8s_util.apply_k8s_specs(model_specs, k8s_util.K8S_CREATE_OR_REPLACE)     

Created Deployment anonymous.mnist-model
Created Service anonymous.mnist-service
Deleted ConfigMap anonymous.mnist-model
Created ConfigMap anonymous.mnist-model


[{'api_version': 'apps/v1',
  'kind': 'Deployment',
  'metadata': {'annotations': None,
               'cluster_name': None,
               'creation_timestamp': datetime.datetime(2020, 3, 31, 11, 3, 8, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': 1,
               'initializers': None,
               'labels': {'app': 'mnist'},
               'managed_fields': None,
               'name': 'mnist-model',
               'namespace': 'anonymous',
               'owner_references': None,
               'resource_version': '19343303',
               'self_link': '/apis/apps/v1/namespaces/anonymous/deployments/mnist-model',
               'uid': '34164356-733f-11ea-bedf-42010aa00fec'},
  'spec': {'min_ready_seconds': None,
           'paused': None,
           'progress_deadline_seconds': 600,
           'replicas': 1,
 

## Deploy the UI for the MNIST web app

Deploy the UI to visualize the MNIST prediction results.

This example uses a prebuilt and public Docker image for the UI.

In [46]:
ui_name = "mnist-ui"
ui_deploy = f"""apiVersion: apps/v1
kind: Deployment
metadata:
  name: {ui_name}
  namespace: {namespace}
spec:
  replicas: 1
  selector:
    matchLabels:
      app: mnist-web-ui
  template:
    metadata:
      labels:
        app: mnist-web-ui
    spec:
      containers:
      - image: gcr.io/kubeflow-examples/mnist/web-ui:v20190112-v0.2-142-g3b38225
        name: web-ui
        ports:
        - containerPort: 5000
        hosts:
        - '*'
      serviceAccount: default-editor
"""

ui_service = f"""apiVersion: v1
kind: Service
metadata:
  annotations:
  name: {ui_name}
  namespace: {namespace}
spec:
  ports:
  - name: http-mnist-ui
    port: 80
    targetPort: 5000
  selector:
    app: mnist-web-ui
  type: ClusterIP
"""

ui_virtual_service = f"""apiVersion: networking.istio.io/v1alpha3
kind: VirtualService
metadata:
  name: {ui_name}
  namespace: {namespace}
spec:
  gateways:
  - kubeflow/kubeflow-gateway
  hosts:
  - '*'
  http:
  - match:
    - uri:
        prefix: /mnist/{namespace}/ui/
    rewrite:
      uri: /
    route:
    - destination:
        host: {ui_name}.{namespace}.svc.cluster.local
        port:
          number: 80
    timeout: 300s
"""

ui_specs = [ui_deploy, ui_service, ui_virtual_service]

In [47]:
k8s_util.apply_k8s_specs(ui_specs, k8s_util.K8S_CREATE_OR_REPLACE)     
    

Created Deployment anonymous.mnist-ui
Created Service anonymous.mnist-ui
Deleted VirtualService anonymous.mnist-ui
Created VirtualService mnist-ui.mnist-ui


[{'api_version': 'apps/v1',
  'kind': 'Deployment',
  'metadata': {'annotations': None,
               'cluster_name': None,
               'creation_timestamp': datetime.datetime(2020, 3, 31, 11, 3, 16, tzinfo=tzlocal()),
               'deletion_grace_period_seconds': None,
               'deletion_timestamp': None,
               'finalizers': None,
               'generate_name': None,
               'generation': 1,
               'initializers': None,
               'labels': None,
               'managed_fields': None,
               'name': 'mnist-ui',
               'namespace': 'anonymous',
               'owner_references': None,
               'resource_version': '19343371',
               'self_link': '/apis/apps/v1/namespaces/anonymous/deployments/mnist-ui',
               'uid': '38fb721b-733f-11ea-bedf-42010aa00fec'},
  'spec': {'min_ready_seconds': None,
           'paused': None,
           'progress_deadline_seconds': 600,
           'replicas': 1,
           'revisi

## Access the MNIST web UI

A reverse proxy route is automatically added to the Kubeflow IAP endpoint. The MNIST endpoint is:

  ```
  https:/${KUBEFlOW_ENDPOINT}/mnist/${NAMESPACE}/ui/  
  ```
  
where `NAMESPACE` is the namespace where you're running the Jupyter notebook.

In [48]:
if endpoint:    
    vs = yaml.safe_load(ui_virtual_service)
    path= vs["spec"]["http"][0]["match"][0]["uri"]["prefix"]
    ui_endpoint = endpoint + path
    display(HTML(f"mnist UI is at <a href='{ui_endpoint}'>{ui_endpoint}</a>"))


Open the MNIST UI in your browser. You should see an image of a hand-written digit from 0 to 9. This is a random image sent to the model for classification. Below the image is a set of bar graphs, one for each classification label from 0 to 9, as output by the model. Each bar represents the probability that the image matches the respective label. 

Click the **test random image** button to send the model a new image.

## Next steps

Visit the [Kubeflow docs](https://www.kubeflow.org/docs/gke/) for more information about running Kubeflow on GCP.